In [1]:
import os
import PyPDF2
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain_groq import ChatGroq
from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.enums import TA_JUSTIFY


c:\Users\PRECISION\AppData\Local\Programs\Python\Python311\Lib\site-packages\pydantic\_internal\_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [2]:
# Set up Groq API key
os.environ["GROQ_API_KEY"] = ""  # Replace with your actual Groq API key

In [3]:
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
    return text

In [4]:
def create_qa_system(past_paper_paths, subject_book_paths):
    all_texts = ""
    for pdf_path in past_paper_paths + subject_book_paths:
        pdf_text = extract_text_from_pdf(pdf_path)
        all_texts += pdf_text + "\n"

    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    texts = text_splitter.split_text(all_texts)

    embeddings = HuggingFaceEmbeddings()
    db = Chroma.from_texts(texts, embeddings)

    llm = ChatGroq(
        model="llama-3.1-70b-versatile",
        temperature=0.7,
    )

    qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=db.as_retriever(search_kwargs={"k": 3}))

    return qa

In [5]:
def generate_question_paper(qa_system, num_questions):
    questions = []
    for i in range(num_questions):
        prompt = f"Generate a new multiple choice exam which has 10 questions based on the content of the past papers, ensuring it's within the scope of the subject books. The question should be challenging but fair. Format the question in a structured manner suitable for an exam paper, including clear instructions. give mcqs in the form of a,b,c,d options. also space between each question should be at least 2 lines in proper format."
        response = qa_system.run(prompt)
        questions.append(response)
    return questions

In [6]:
def create_reformatted_question_paper_pdf(questions, output_path):
    doc = SimpleDocTemplate(output_path, pagesize=letter)
    styles = getSampleStyleSheet()
    content = []

    content.append(Paragraph("Generated Question Paper", styles['Title']))
    content.append(Spacer(1, 12))

    styles.add(ParagraphStyle(name='Justify', alignment=TA_JUSTIFY))
    
    for i, question in enumerate(questions, 1):
        content.append(Paragraph(f"Question {i}", styles['Heading2']))
        lines = question.split('\n')
        for line in lines:
            if line.strip():
                if line.startswith('*') or line.startswith('-'):
                    content.append(Paragraph(f"• {line.strip('*- ')}", styles['BodyText']))
                else:
                    content.append(Paragraph(line, styles['Justify']))
        content.append(Spacer(1, 12))

    doc.build(content)

In [7]:
import os

def get_pdf_files_from_folder(folder_path):
    """
    Returns a list of PDF file paths from the specified folder.
    """
    pdf_files = []
    for file in os.listdir(folder_path):
        if file.lower().endswith('.pdf'):
            pdf_files.append(os.path.join(folder_path, file))
    return pdf_files



In [8]:
def main():
    past_paper_folder = "mcqs_output_pdfs"  # Folder containing past paper PDFs
    past_paper_paths = get_pdf_files_from_folder(past_paper_folder)
    
    subject_book_paths = ["cs9.pdf"]  # Add your subject book PDFs here
    
    qa_system = create_qa_system(past_paper_paths, subject_book_paths)
    
    num_questions = 1  # You can adjust this number
    questions = generate_question_paper(qa_system, num_questions)
    
    output_pdf_path = "multiple choice questions.pdf"
    create_reformatted_question_paper_pdf(questions, output_pdf_path)
    
    print(f"Generated question paper has been saved to {output_pdf_path}")
    print(f"Used {len(past_paper_paths)} past papers from the folder '{past_paper_folder}'")

if __name__ == "__main__":
    main()

Created a chunk of size 26485, which is longer than the specified 1000
C:\Users\PRECISION\AppData\Local\Temp\ipykernel_17080\4250945908.py:10: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings()
C:\Users\PRECISION\AppData\Local\Temp\ipykernel_17080\4250945908.py:10: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()
c:\Users\PRECISION\AppData\Local\Programs\Python\Python311\Lib\site-packages\sentence_transformers

Generated question paper has been saved to multiple choice questions.pdf
Used 33 past papers from the folder 'output_pdfs'


In [9]:
# def main():
#     past_paper_paths = ["output.pdf", "output1.pdf"]  # Your past paper PDFs
#     subject_book_paths = ["cs10.pdf"]  # Add your subject book PDFs here
    
#     qa_system = create_qa_system(past_paper_paths, subject_book_paths)
    
#     num_questions = 10  # You can adjust this number
#     questions = generate_question_paper(qa_system, num_questions)

#     output_pdf_path = "r1_generated_question_paper.pdf"
#     create_reformatted_question_paper_pdf(questions, output_pdf_path)
#     print(f"Generated question paper has been saved to {output_pdf_path}")

# if __name__ == "__main__":
#     main()